# DM (device manufacturer) connects to music (device) and assign them ownership - DM

## 1. Create Invitation from agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections


## 2. Create invitation

In [2]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection ID 3a5b59b0-f221-429b-b8f3-b121e1b7c640
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '2065f3a1-1568-41ad-9a0d-d217d3eecfd0', 'recipientKeys': ['2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice'}
Connection Handler Called
Connection 3a5b59b0-f221-429b-b8f3-b121e1b7c640 in State invitation
Connection Handler Called
Connection 3a5b59b0-f221-429b-b8f3-b121e1b7c640 in State request


## 4. Accept Request - Recipient first!!!!!

In [3]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 3a5b59b0-f221-429b-b8f3-b121e1b7c640 in State response
ACCEPT REQUEST
{'invitation_key': '2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y', 'updated_at': '2023-01-03 12:11:55.955925Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:10:47.835278Z', 'my_did': 'XtNBn5VgqaouerjBv9zGiy', 'their_did': 'JwZFj9aDdiL1d4pJSSL3Wj', 'state': 'response', 'their_label': 'Bob', 'connection_id': '3a5b59b0-f221-429b-b8f3-b121e1b7c640', 'rfc23_state': 'response-sent', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}
state response


In [4]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
connection_id = connection["connection_id"]

Trust Ping {'thread_id': '97c6e4da-41ac-4dbd-bbe9-183416d83b91'}
{'invitation_key': '2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y', 'updated_at': '2023-01-03 12:11:55.955925Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:10:47.835278Z', 'my_did': 'XtNBn5VgqaouerjBv9zGiy', 'their_did': 'JwZFj9aDdiL1d4pJSSL3Wj', 'state': 'response', 'their_label': 'Bob', 'connection_id': '3a5b59b0-f221-429b-b8f3-b121e1b7c640', 'rfc23_state': 'response-sent', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}
Is Active? response
Connection Handler Called
Connection 3a5b59b0-f221-429b-b8f3-b121e1b7c640 in State active


# Give MUSIC the VC of ownership and origin - DM

## 1. Write Ownership schema

In [5]:
# Define you schema name - must be unique on the ledger
schema_name = "music_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["origin", "name", "time", "version", "author", "owner"]

# Write schema to ledger
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

# Write cred. def.
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)


# # Update schema
# attributes = ["author", "name", "time", "is_original", "type"]
# schema_version = "0.0.2"
# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# updated_schema_id = response["schema_id"]
# print(updated_schema_id)

# # Write another Credential Definition for the Updated Schema
# response = await agent_controller.definitions.write_cred_def(updated_schema_id, support_revocation=True)
# updated_cred_def_id = response["credential_definition_id"]
# print(updated_cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1
PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default


## 2. Write VC

In [6]:
nym = await agent_controller.wallet.get_public_did()
print(nym["result"]["did"])
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did='PQRXDxdGqQGSZ8z69p4xZP')
print(created_schema)
schema_id = created_schema["schema_ids"][-1]

PQRXDxdGqQGSZ8z69p4xZP
{'schema_ids': ['PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1']}


In [7]:
definitions = await agent_controller.definitions.search_created(schema_id=schema_id)
print(definitions["credential_definition_ids"])
cred_def_id = definitions["credential_definition_ids"][-1]

['PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default']


In [8]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results']
    for con in connection:
        print("Connection :", con)
        if con['state'] == 'active':       
            connection_id = con["connection_id"]
            print("Active Connection ID : ", connection_id)
        else: print("Inactive Connection ID: ", con["connection_id"], "with state: ", con["state"])
else:
    print("You must create a connection")

1
Connection : {'invitation_key': '2EZgL2gMffbxQFWBS2mVkuLmFPAzaMVgwGZXTswYCE6y', 'updated_at': '2023-01-03 12:11:59.431606Z', 'invitation_mode': 'once', 'created_at': '2023-01-03 12:10:47.835278Z', 'my_did': 'XtNBn5VgqaouerjBv9zGiy', 'their_did': 'JwZFj9aDdiL1d4pJSSL3Wj', 'state': 'active', 'their_label': 'Bob', 'connection_id': '3a5b59b0-f221-429b-b8f3-b121e1b7c640', 'rfc23_state': 'completed', 'their_role': 'invitee', 'routing_state': 'none', 'accept': 'manual'}
Active Connection ID :  3a5b59b0-f221-429b-b8f3-b121e1b7c640


In [11]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": nym["result"]["did"]},
    {"name": "name", "value": "balabababa music"},
    {"name": "time", "value": time},
    {"name": "origin", "value": nym["result"]["did"]},
    {"name": "owner", "value": nym["result"]["did"]},
    {"name": "version", "value": "v1"}
]
print(credential_attributes)


# send credential
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

[{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'name', 'value': 'balabababa music'}, {'name': 'time', 'value': '01-03-2023, 12:13:19'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'version', 'value': 'v1'}]
Credential exchange c5dc49c8-0dd3-43ad-846c-8b9d24754ed1, role: issuer, state: offer_sent


## 4. Wait for credential exchange

In [10]:
response = await agent_controller.issuer.get_records()
print(response['results'])
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

[{'credential': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default', 'rev_reg_id': None, 'values': {'author': {'raw': 'PQRXDxdGqQGSZ8z69p4xZP', 'encoded': '43137799088118443800136745895388480458686925529355594539512585697577946794869'}, 'version': {'raw': 'v1', 'encoded': '27131969825814800887461230010484753141979851386045457649691352597878165178622'}, 'owner': {'raw': 'PQRXDxdGqQGSZ8z69p4xZP', 'encoded': '43137799088118443800136745895388480458686925529355594539512585697577946794869'}, 'time': {'raw': '01-03-2023, 12:12:33', 'encoded': '48587442510936264572122896078570777346685050877026407568254355401140050668337'}, 'name': {'raw': 'balabababa music', 'encoded': '78349233637792322947684343969666349239482962495698159244268994899549966660302'}, 'origin': {'raw': 'PQRXDxdGqQGSZ8z69p4xZP', 'encoded': '43137799088118443800136745895388480458686925529355594539512585697577946794869'}}, 'signature': {'p_credential': {'m_2': '16777

In [24]:
## Remove records
response = await agent_controller.issuer.remove_record(record_id)
print(response)

{}


In [12]:
response = await agent_controller.terminate()
print(response)

None
